In [269]:
# Trying to combine the DFM aspects of dfm_rs_and_chl_runs
# and the age/nitrate/etc. aspects of run_tracers_age

In [265]:
import sys
sys.path.append("/richmondvol1/rusty/stompy")

In [266]:
import os, glob, shutil
import xml.etree.ElementTree as ET
import datetime
import six
import xarray as xr
import pandas as pd

In [270]:
from stompy.spatial import field
from stompy import utils
from shapely import geometry

import netCDF4
import xarray as xr
import stompy.model.delft.dflow_model as dfm
import stompy.model.delft.waq_scenario as dwaq
from stompy.grid import unstructured_grid
import matplotlib.pyplot as plt
import logging as log
from scipy.interpolate import griddata
from stompy.spatial import proj_utils
import numpy as np
%matplotlib notebook

In [271]:
import bloom_common
scene_df = bloom_common.load_chl_scenes()

In [272]:
from bloom_common import load_scene, load_scene_utm

In [198]:
six.moves.reload_module(dwaq)

# with dfm_spinup in place, no need to go back to this original run.

#if 1: # longer, 16 layer run.
#    # The DFM setup that we're repurposing
#    dfm_path="/chicagovol1/hpcshared/open_bay/hydro/full_res/wy2022_bloom/runs/wy2022_bloom_16layer"    
#    #dwaq_hydro=os.path.join(dfm_path, "DFM_DELWAQ_wy2022_bloom_16layer")
#    #hydro=dwaq.HydroFiles(os.path.join(dwaq_hydro,"wy2022_bloom_16layer.hyd"))

bloom_common.configure_dfm_t140737() # broken for dwaq, but hopefully works for dfm-online-dwaq
#bloom_common.configure_dfm_2023_01() # used to be configure_dwaq_new()
    

DFM-based Runs
==

In [276]:
# restart files every 10 days, plus a restart file on 2022-08-01.
# speed was 85x realtime on 16 cores.
# Forcing should all be okay to copy/link for restarts.

if 1: # Check on map and rst output:
    dfm_base_run_dir="dfm_spinup"
    map_fn=os.path.join(dfm_base_run_dir,
                        "DFM_OUTPUT_wy2022_bloom_16layer",
                        "wy2022_bloom_16layer_0000_20220501_000000_map.nc")
    rst_fn=os.path.join(dfm_base_run_dir,
                        "DFM_OUTPUT_wy2022_bloom_16layer",
                        "wy2022_bloom_16layer_0000_20220801_000000_rst.nc")
    
#map_ds=xr.open_dataset(map_fn)
#rst_ds=xr.open_dataset(rst_fn)

In [284]:
six.moves.reload_module(dfm)
six.moves.reload_module(bloom_common)
import pdb

# First, get a basic restart going
class SFBRestartable(dfm.DFlowModel):
    """
    Add special sauce to symlink relevant forcing files that are
    not standard for dflow_model.
    This includes bc_files, src_files, meteo_coarse.grd
    """
    restart_copy_names=["source_files"] # symlink everything

    def copy_files_for_restart(self):
        super().copy_files_for_restart()
        prev_model=self.restart_from
        # Subdirectories are not automatically copied over, as well as meteo_coarse.grd
        for sub in ['bc_files','source_files','meteo_coarse.grd']:
            src=os.path.join(prev_model.run_dir,sub)
            if not os.path.exists(src):
                self.log.warning(f"Expected to symlink {src} but it wasn't there.")
                continue
            dst=os.path.join(self.run_dir,sub)
            if os.path.exists(dst):
                self.log.warning(f"Expected to make {dst} a copy or symlink but it already exists")
                continue

            if sub in self.restart_copy_names:
                self.log.info(f"Copy {src} => {dst}")
                shutil.copytree(src,dst)
            else:
                # TODO: chase down symlinks to get back to the real file
                # otherwise this will break if intermediate runs are removed.
                # Make the symlinks relative in all of this moves, is on a different machine, etc.
                src_rel=os.path.relpath(src,start=self.run_dir)
                self.log.info(f"Symlink {dst} => {src_rel}")
                os.symlink(src_rel,dst)        

prev_model=SFBRestartable.load(dfm_base_run_dir)


# And the remote sensed imagery part 
class AgeDfm:
    name="data_agedfm_v00"
    run_dir_prefix="run" # {name}/{run_dir_prefix}_{dates}_{version}

    # Will start from the end of this existing run (which doesn't have the tracers)
    dfm_base_run_dir="dfm_spinup"
    # run_dir that gets us up to run_start
    pre_run_dir=None
    
    # spinup will be run without tracers up to this date:
    run_start=np.datetime64("2022-08-08")
    run_stop =np.datetime64("2022-08-30")
    
    #tracers_per_speed=['weight','wvalue','uniform']
    swim_speeds=[0.0] # positive down, m/s
    
    restart_copy_names=["source_files"] # copy, because we end up modifying some
    
    initial_condition='mixed' # only support mixed right now
    
    polygons={'oakland':bloom_common.oakland_poly,
              'eastshore':bloom_common.eastshore_poly}

    # HERE - copy in the tracer setup.
    # Maybe move custom tracer code to bloom_common.
    
    def run_simulations(self):
        """
        Find a restart point close to start_scene_idx, 
        initialize, run to stop_scene_idx. If start_scene_idx is
        negative, skip tracer setup, just trying to get a restart file
        that matches stop_scene_idx, and assuming that the last 
        """
        self.run_to_start()
        # self.run_start_to_stop()
        
    def run_to_start(self):
        prev_model=SFBRestartable.load(self.dfm_base_run_dir)            
        restart_time=prev_model.restartable_time()
        
        assert restart_time < self.run_start
        
        # Setup a restart and then check whether it already exists
        # before actually running it.
        model=prev_model.create_restart(deep=True)
        model.run_stop=self.run_start # we're just trying to get up to the start
        self.set_run_dir(model)
        
        self.pre_run_dir=model.run_dir
        
        if dfm.DFlowModel.run_completed(model.run_dir):
            print("Run to start time is already complete")
            return

        # no tracers for this part
        # though if we start adding swimming and want a good IC,
        # here is where a uniform+swimming tracer could be added.
        # self.set_scene_tracers(model,start_scene_idx)
            
        self.generally_configure(model)
        # This alters the MDU, so do it before write()
        # self.update_restart_with_tracers(model)
        model.write()
        # This updates the BC data in place. Do it here so that 
        # we have a starting ext file which will be updated with
        # new tracers.
        # self.add_tracers_to_bcs(model,self.tracers)
        model.partition()
        model.run_simulation()

    def run_start_to_stop(self):
        prev_model=SFBRestartable.load(self.pre_run_dir)            

        # Setup a restart
        model=prev_model.create_restart(deep=True)
        model.run_stop=self.run_stop
        
        self.set_run_dir(model)

        self.set_scene_tracers(model)
            
        self.generally_configure(model)
        # This alters the MDU, so do it before write()
        self.update_restart_with_tracers(model)
        model.write()
        # This updates the BC data in place. Do it here so that 
        # we have a starting ext file which will be updated with
        # new tracers.
        self.add_tracers_to_bcs(model,self.tracers)
        model.partition()
        model.run_simulation()

    def generally_configure(self,model):
        model.mdu['output','WaqInterval']="" # no need for DWAQ output

        bloom_common.configure_dfm_t140737()
        model.dfm_bin_dir=os.path.join(os.environ['DELFT_SRC'],'bin')
        model.mpi_bin_dir=os.path.join(os.environ['DELFT_SRC'],'bin')
        
        bloom_common.set_minimal_map_output(model)
        
        model.mdu['output','MapInterval']="900" # make some nice animations

    
    def set_run_dir(self,model):
        start_str,stop_str=[ utils.to_datetime(t).strftime("%Y%m%dT%H%M")
                            for t in [model.run_start,model.run_stop]]
        for x in range(20):
            run_dir=os.path.join(self.name,f"{self.run_dir_prefix}_{start_str}_{stop_str}_v{x:02}")
            if not os.path.exists(run_dir): break
        else:
            raise Exception(f"Too many retries for {run_dir}")
        model.run_dir=run_dir
        model.set_restart_file() # kludge. RestartFile needs run_dir.
    
    def set_scene_tracers(self,model,start_scene_idx):
        # Replace the RS-based code with simple polygon blob
        # Allow multiple swimming speeds but don't bother with stratified IC.
        assert self.initial_condition=='mixed'
        tracers=[]
    
        for swim_i,swim_speed in enumerate(self.swim_speeds): # positive down.
            for tracer_type in self.tracers_per_speed:
                # HERE - adapt with the age tracers
                assert tracer_type in self.polygons
                tracer_name=tracer_type+str(swim_i)
                
                def tracer_func(rst_ds,values_cell_layer,tracer_type=tracer_type,
                                incoming_uniform=None,swim_i=swim_i):
                    if swim_speed!=0.0 and self.initial_condition=='stratified':
                        vert_scale = incoming_uniform[swim_i]
                    else:
                        vert_scale=1.0
                    # vert_scale is guaranteed to have unit concentration in the surface
                    # layer of all cells.

                    polygon=self.polygons[tracer_type]                    
                    xy=np.c_[ rst_ds.FlowElem_xzw.values, rst_ds.FlowElem_yzw.values]

                    cells=[polygon.contains(geometry.Point(x,y)) 
                           for x,y in zip(rst_ds.FlowElem_xzw.values, rst_ds.FlowElem_yzw.values)]
                    value_2d=100*np.array(cells,np.float64)
                    values_cell_layer[:,:] = value_2d[:,None]

                tracers.append( dict(name=tracer_name,func=tracer_func,
                                     fall_velocity_m_s=swim_speed))            
            
        self.tracers=tracers

    def update_restart_with_tracers(self,model):
        def modify_ic(rst_ds,**kw):
            assert self.initial_condition=='mixed'
            incoming_uniform=None
            
            for tracer in self.tracers:
                name=tracer['name']
                func=tracer['func']
                model.log.info(f"Setting tracer {name} in restart file")
                # mimic sa1 tracer
                salt=rst_ds['sa1']
                values=salt.values.copy() # ('time','nFlowElem','laydim')
                values[...] = 0.0 # don't accidentally write salt data though
                
                # updates values in place.
                func(rst_ds=rst_ds,values_cell_layer=values[0,:,:], 
                     incoming_uniform=incoming_uniform)
                rst_ds[name]=salt.dims, values
                for aname in ['coordinates','grid_mapping']:
                    if aname in salt.attrs:
                        rst_ds[name].attrs[aname]=salt.attrs[aname]
        model.modify_restart_data(modify_ic)                

    def add_tracers_to_bcs(self,model,tracers):
        # take a more low-level approach compared to usual BC configuration
        # so that we can be very careful about what things change.
        ext_fn=model.mdu.filepath(('external forcing','ExtForceFile'))
        orig_ext_fn=ext_fn+".orig"
        # This check is more for dev -- it is fragile in the sense that if
        # orig_ext_fn *should* be different, we'll end up still using the
        # old file.
        if not os.path.exists(orig_ext_fn):
            shutil.copyfile(ext_fn,orig_ext_fn)

        bcs=model.parse_old_bc(orig_ext_fn)
        
        new_tracer_names=[t['name'] for t in tracers]
        configured_tracers={}
        
        # For now all boundary conditions for all new tracers are 0.
        # Note that establishing order here is very confusing. If these
        # need to be nonzero, it will take some work to know that 
        # it's correct. probably the strategy should be to filter out 
        # all existing BCs for these tracers, and then write them at the
        # end in our prescribed order. Yeah, that's what I'm doing.
        new_bc_values=[0.0 for t in tracers]

        def name_matches(cfg_name):
            for tracer in tracers:
                if tracer['name'].lower() == cfg_name.lower():
                    if tracer['name']!=cfg_name:
                        print(f"Careful - case mismatch {cfg_name} vs {tracer['name']}")
                    return True
            return False
            
        with open(ext_fn,'wt') as fp_new:
            for rec in bcs:
                write_verbatim=True
                
                quantity=rec['QUANTITY']
                if quantity.upper().startswith('INITIALTRACER'):
                    tracer_name=quantity[len("INITIALTRACER"):]
                    #if name_matches(tracer_name): continue
                    # Assume that we will rewrite *all* tracers.
                    # otherwise we have to keep track of how many tracers
                    # are along for the ride in addition to the ones we're
                    # adding.
                    continue
                    # Tracer ICs we care about just because they help establish the
                    # list of tracers, but no need to change these entries.
                    # Actually, will just write out fresh stanzas for these in order
                    # to force the ordering.
                    # configured_tracers[tracer_name]=True
                elif quantity.upper().startswith('TRACERBND'):
                    tracer_name=quantity[len("TRACERBND"):]
                    #if name_matches(tracer_name): continue
                    continue # as above.
                    # And for now we leave boundary conditions as is, but again
                    # remember that this tracer has been configured.
                    #configured_tracers[tracer_name]=True
                elif quantity.upper().startswith('DISCHARGE_SALINITY_TEMPERATURE_SORSIN'):
                    print("Source/sink BC entry")
                    # Yuck - have to add new column(s). This only involves rewriting 
                    # the data file,though. The stanza is unchanged.
                    self.add_tracer_bcs(model,rec,new_values=new_bc_values,orig_num_values=3)

                # At this point nobody every changes the stanza, it's all written verbatim.
                if write_verbatim:
                    fp_new.write("\n".join(rec['stanza'])+"\n")
                    continue
                
            # And write out our new tracers (including ones that were skipped during 
            # transcription above
            for tracer in self.tracers:
                name=tracer['name']
                ic_fn=f"dummy-{name}.xyz"
                with open(os.path.join(model.run_dir,ic_fn),'wt') as fp_xyz:
                    fp_xyz.write("550000 4180000 0.0\n")
                fp_new.write("\n# NEW TRACERS\n"
                             f"QUANTITY=initialtracer{name}\n"
                             f"FILENAME={ic_fn}\n"
                             "FILETYPE=7\n"
                             "METHOD=5\n"
                             "OPERAND=O\n")
                if tracer['fall_velocity_m_s']!=0.0:
                    w=tracer['fall_velocity_m_s']
                    fp_new.write(f"TRACERFALLVELOCITY={w:.8f}\n")

    def add_tracer_bcs(self,model,bc,new_values=[],orig_num_values=3):
        """
        Add additional columns to a source/sink data file.
        So if the new run will include two dwaq tracers, pass new_values=[0,1]
        (which would tag sources with 0 for the first and 1.0 for the second)
        orig_num_values: 3 for run with salinity and temperature. I think
        less than that if temperature and/or salinity are disabled. 
        """
        # yuck...
        pli_fn=os.path.join(model.run_dir,bc['FILENAME'])
        assert pli_fn.lower().endswith('.pli')
        fn=pli_fn[:-4] + ".tim"
        assert os.path.exists(fn)
        fn_orig=fn+".orig"
        if not os.path.exists(fn_orig):
            shutil.copyfile(fn,fn_orig)
        data_orig=np.loadtxt(fn_orig)
        # drop previous forcing for new tracers. leaving time column and the original Q,S,T values
        columns=[data_orig[:,:1+orig_num_values]] 
        for new_val in new_values:
            columns.append( np.full(data_orig.shape[0],new_val))
        data=np.column_stack(columns)
        np.savetxt(fn,data,fmt="%.6g")
        
        
# Bringing in the age-tracer setup
# Exactly what tracers do I even care about?
# Most basic - could use the existing blob runs along with exponential growth.
# Next would be to cap at some concentration related to nutrient limitation.
# there were runs that also 

In [285]:
agedfm=AgeDfm() 
agedfm.run_simulations()

INFO:HydroModel:set_restart_file: Setting RestartFile based on self.restart_from
INFO:HydroModel:set_restart_file: Setting RestartFile based on self.restart_from
INFO:HydroModel:Could not find BC to get initial water level
INFO:DFlowModel:Writing MDU to data_agedfm_v00/run_20220801T0000_20220808T0000_v00/wy2022_bloom_16layer.mdu
INFO:HydroModel:Symlink data_agedfm_v00/run_20220801T0000_20220808T0000_v00/bc_files => ../../dfm_spinup/bc_files
INFO:HydroModel:Copy dfm_spinup/source_files => data_agedfm_v00/run_20220801T0000_20220808T0000_v00/source_files
INFO:HydroModel:Symlink data_agedfm_v00/run_20220801T0000_20220808T0000_v00/meteo_coarse.grd => ../../dfm_spinup/meteo_coarse.grd
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0000_net.nc => data_agedfm_v00/run_20220801T0000_20220808T0000_v00/sfei_v20_0000_net.nc
INFO:HydroModel:Copying pre-partitioned grid files: dfm_spinup/sfei_v20_0001_net.nc => data_agedfm_v00/run_20220801T0000_20220808T0000_v00/sfei_v20_0001

CalledProcessError: Command '['/opt/software/delft/dfm/t140737/bin/mpiexec', '-n', '16', '/opt/software/delft/dfm/t140737/bin/dflowfm', '-t', '1', '--autostartstop', 'wy2022_bloom_16layer.mdu']' returned non-zero exit status 1.